<a href="https://colab.research.google.com/github/dainesjeff/MLFundamentals/blob/main/ML_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **3D Linear Regression (Pseudoinverse Implementation)**

# **Tensorflow**

In [ ]:
import tensorflow as tf

## Add checks for Data Type and shape


def regress_with_puedoinverse(X,b):
  pseudoinverse_X = tf.linalg.pinv(X)
  return pseudoinverse_X @ b
X = tf.random.uniform([10,3],minval=0, maxval = 10)
b = tf.linspace([1], [10], 10, name=None, axis=0)
b= tf.cast(b, X.dtype)
a = regress_with_puedoinverse(X,b)

print(X.dtype)
print(b.shape)
print(a)

<dtype: 'float32'>
(10, 1)
tf.Tensor(
[[ 0.84599555]
 [ 0.549078  ]
 [-0.7176521 ]], shape=(3, 1), dtype=float32)


# **Pytorch**

In [ ]:
import torch

torch.manual_seed(43)

def regress_with_puedoinverse(X,b):
  pseudoinverse_X = torch.linalg.pinv(X)
  return pseudoinverse_X @ b

X = torch.rand(10,3) * 10
b = torch.linspace(1,10, steps =10, dtype=X.dtype)
b = torch.reshape(b, (10, 1))
a = regress_with_puedoinverse(X,b)
print(a)




tensor([[0.3973],
        [0.6357],
        [0.1045]])


## **Fully Connected MLP**

In [ ]:
%pip install ucimlrepo

from ucimlrepo import fetch_ucirepo
import pandas as pd


# fetch dataset
wine = fetch_ucirepo(id=109)

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf




# targets = pd.get_dummies(wine.data.targets['class'])
# print(targets.head())

features = wine.data.features.to_numpy()
targets = wine.data.targets.to_numpy()



train_X, test_X, train_y, test_y = train_test_split(features, targets,test_size=0.2, train_size=0.8)

train_y = tf.keras.utils.to_categorical(train_y)
test_y = tf.keras.utils.to_categorical(test_y)


In [ ]:
import tensorflow as tf


model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_dim=13))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2), metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1792      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 12260 (47.89 KB)
Trainable params: 12260 (47.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
history = model.fit(x= train_X, y = train_y, batch_size=16, epochs=16, validation_data = (test_X, test_y))

Epoch 1/16
9/9 [==============================] - 13s 63ms/step - loss: 133.9157 - accuracy: 0.3099 - val_loss: 3.7922 - val_accuracy: 0.3056
Epoch 2/16
9/9 [==============================] - 0s 12ms/step - loss: 1.5317 - accuracy: 0.4507 - val_loss: 1.2093 - val_accuracy: 0.3889
Epoch 3/16
9/9 [==============================] - 0s 10ms/step - loss: 1.1872 - accuracy: 0.4014 - val_loss: 1.1177 - val_accuracy: 0.3889
Epoch 4/16
9/9 [==============================] - 0s 11ms/step - loss: 1.1115 - accuracy: 0.3239 - val_loss: 1.1711 - val_accuracy: 0.2222
Epoch 5/16
9/9 [==============================] - 0s 14ms/step - loss: 1.1098 - accuracy: 0.3592 - val_loss: 1.1171 - val_accuracy: 0.3889
Epoch 6/16
9/9 [==============================] - 0s 12ms/step - loss: 1.0888 - accuracy: 0.4014 - val_loss: 1.1050 - val_accuracy: 0.3889
Epoch 7/16
9/9 [==============================] - 0s 11ms/step - loss: 1.0827 - accuracy: 0.3451 - val_loss: 1.1387 - val_accuracy: 0.3889
Epoch 8/16
9/9 [========

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('epochs', fontsize=15)
plt.legend(fontsize=20)
plt.show()
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('epochs', fontsize=15)
plt.legend(fontsize=20)
plt.show()

NameError: ignored

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class DenseNet(nn.Module):
  def __init__(self):
    super(DenseNet, self).__init__()
    self.fc1 = nn.Linear(13, 32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,32)
    self.out = nn.Linear(32, 4)

    self.linear_relu_stack = nn.Sequential(
    nn.Linear(13, 32),
    nn.ReLU(),
    nn.Linear(32, 64),
    nn.ReLU(),
    nn.Linear(64, 4)
    )

  def forward(self, x):
      logits = self.linear_relu_stack(x)
      return logits

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

model = DenseNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

batch_size = 10
epochs = 16


torch_train_X =  Variable(torch.from_numpy(train_X).float(), requires_grad=False)
torch_train_y = Variable(torch.from_numpy(train_y), requires_grad=False)
train_dataset = torch.utils.data.TensorDataset(torch_train_X, torch_train_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

for i in range(epochs):
  train(train_dataloader, model, loss_fn, optimizer)



NameError: ignored